In [1]:
%run _spark_init_.ipynb

- Write a structured query that “transpose” a dataset so a new dataset uses column names and values from a struct column.

In [40]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, MapType

critics_data = [("Manuel", [["Logan", 1.5], ["Zoolander", 3.0], ["John Wick", 2.5]]),
                ("John", [["Logan", 2.0], ["Zoolander", 3.5], ["John Wick", 3.0]])]

schema = StructType([StructField("name", StringType(), True),
                     StructField("movie_ratings", ArrayType(StructType([StructField("movie_name", StringType(), True),
                                                                        StructField("rating", DoubleType(), True)])), True)])
critics_df = spark.createDataFrame(critics_data, schema)

In [41]:
critics_df.show(truncate=False)

+------+--------------------------------------------------+
|name  |movie_ratings                                     |
+------+--------------------------------------------------+
|Manuel|[{Logan, 1.5}, {Zoolander, 3.0}, {John Wick, 2.5}]|
|John  |[{Logan, 2.0}, {Zoolander, 3.5}, {John Wick, 3.0}]|
+------+--------------------------------------------------+



In [56]:
from pyspark.sql.functions import first

(critics_df
    .selectExpr("name","explode(movie_ratings) as mr")
    .select("name", "mr.movie_name", "mr.rating")
    .groupBy("name")
    .pivot("movie_name")
    .agg(first("rating"))
    .show()
)

+------+---------+-----+---------+
|  name|John Wick|Logan|Zoolander|
+------+---------+-----+---------+
|  John|      3.0|  2.0|      3.5|
|Manuel|      2.5|  1.5|      3.0|
+------+---------+-----+---------+

